### 기본 스태킹 모델

In [1]:
import numpy as np
import pandas as pd

from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings('ignore')

C:\Users\TJ\anaconda3\envs\ml01-02\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [2]:
# 데이터 : 유방암 데이터
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
# 성능평가 지표 : 정확도
from sklearn.metrics import accuracy_score

In [3]:
# 데이터 로딩 / 분리
cancer_data = load_breast_cancer()

# 데이터 추출
X_data = cancer_data.data

# 답 추출
y_label = cancer_data.target

# 분리
X_train, X_test, y_train, y_test =\
train_test_split(
      X_data
    , y_label
    , test_size=0.2
    , random_state=0
)

In [4]:
# 앙상블에 사용할 알고리즘 객체 생성
knn_clf = KNeighborsClassifier(n_neighbors=4)
rf_clf = RandomForestClassifier(n_estimators=100, random_state=0)
dt_clf = DecisionTreeClassifier() # 설정을 따로 안하므로 과적합될 예정
ada_clf = AdaBoostClassifier(n_estimators=100)

In [5]:
# 스태킹에 사용할 객체 생성
lr_clf = LogisticRegression(C=10)

In [9]:
# 개별 학습 진행
knn_clf.fit(X_train, y_train)
rf_clf.fit(X_train, y_train)
dt_clf.fit(X_train, y_train)
ada_clf.fit(X_train, y_train)

AdaBoostClassifier(n_estimators=100)

In [11]:
X_train.shape, X_test.shape

((455, 30), (114, 30))

#### 각 모델의 예측값 추출

In [15]:
knn_pred = knn_clf.predict(X_test)
rf_pred = rf_clf.predict(X_test)
dt_pred = dt_clf.predict(X_test)
ada_pred = ada_clf.predict(X_test)
knn_pred.shape, rf_pred.shape, dt_pred.shape, ada_pred.shape

((114,), (114,), (114,), (114,))

In [17]:
# 각 모델의 예측값 출력
print('KNN 정확도 : {0:.4f}'.format(accuracy_score(y_test, knn_pred)))
print('RF 정확도 : {0:.4f}'.format(accuracy_score(y_test, rf_pred)))
print('DT 정확도 : {0:.4f}'.format(accuracy_score(y_test, dt_pred)))
print('ADA 정확도 : {0:.4f}'.format(accuracy_score(y_test, ada_pred)))

KNN 정확도 : 0.9211
RF 정확도 : 0.9649
DT 정확도 : 0.9035
ADA 정확도 : 0.9561


In [18]:
# 각 모델 예측 결과 합치기
pred = np.array([
      knn_pred
    , rf_pred
    , dt_pred
    , ada_pred
])

In [19]:
pred.shape

(4, 114)

In [20]:
# 전치
pred = np.transpose(pred)
pred.shape

(114, 4)

In [24]:
# 최종 스태킹 진행(학습)
# 학습데이터 : pred
lr_clf.fit(pred, y_test)

# 예측 평가
final = lr_clf.predict(pred)

print(accuracy_score(y_test,final))

0.9736842105263158


### 교차검증(비복원 추출)

In [25]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error # 평균 절대값 오차(MAE) ; 오차구하는함수

In [26]:
# 개별 기반 모델에서 최종 메타 모델이 사용할 학습 및 테스트 생성
def get_stacking_base_datasets(  model     # 개별 기반 모델
                               , X_train_p # 학습데이터
                               , y_train_p # 학습답
                               , X_test_p  # 테스트 데이터
                               , folds_p   # 폴드 개수
                              ):
# 지정된 folds_p 값으로 KFold 객체 생성
    kf = KFold(
          n_splits=folds_p
        , shuffle=False
        , random_state=0
    )
# 최종 모델이 사용할 학습데이터 저장 배열 생성
# 학습데이터 공간확보 : train_fold_pred = np.zeros((행,열))
    train_fold_pred =np.zeros((X_train_p.shape[0],1))

# 테스트 데이터 공간확보
    test_pred = np.zeros((X_test_p.shape[0], folds_p))
    
    print(model.__class__.__name__, ' model 시작')

#   fold.split() 반환 : (train_index, valid_index)
#   folder_counter : 폴드의 반복 개수 저장
    for folder_counter, (train_index, valid_index) in enumerate(kf.split(X_train_p)):
        print('\t 폴드 세트: ', folder_counter, '시작')
#   입력된 학습데이터에서 기반 모델이 학습/예측할 폴드
#   데이터 세트 추출
        X_train_v = X_train_p[train_index] # 학습 데이터 추출
        y_train_v = y_train_p[train_index] # 학습 답 추출
        # 테스트에 사용할 데이터 추출 => 예측값
        X_test_v = X_train_p[valid_index] # 테스트 
        
        # 학습 진행
        model.fit(X_train_v, y_train_v)
        
        # 예측값 추출
        train_fold_pred[valid_index,:] = model.predict(X_test_v).reshape(-1,1)
        
        # 테스트 데이터에 대한 예측값을 저장
        # 예측값이 폴드 개수만큼 추출
        test_pred[:, folder_counter] = model.predict(X_test_p)
        
# for문 end

    test_pred_mean = np.mean(test_pred, axis=1).reshape(-1,1)
    
    return train_fold_pred, test_pred_mean

In [27]:
# KNN
knn_train, knn_test = get_stacking_base_datasets(
      knn_clf
    , X_train
    , y_train
    , X_test
    , 7
)

KNeighborsClassifier  model 시작
	 폴드 세트:  0 시작
	 폴드 세트:  1 시작
	 폴드 세트:  2 시작
	 폴드 세트:  3 시작
	 폴드 세트:  4 시작
	 폴드 세트:  5 시작
	 폴드 세트:  6 시작


In [29]:
# RandomForest
rf_train, rf_test = get_stacking_base_datasets(
      rf_clf
    , X_train
    , y_train
    , X_test
    , 7
)

RandomForestClassifier  model 시작
	 폴드 세트:  0 시작
	 폴드 세트:  1 시작
	 폴드 세트:  2 시작
	 폴드 세트:  3 시작
	 폴드 세트:  4 시작
	 폴드 세트:  5 시작
	 폴드 세트:  6 시작


In [30]:
# DecisionTree
dt_train, dt_test = get_stacking_base_datasets(
      dt_clf
    , X_train
    , y_train
    , X_test
    , 7
)

DecisionTreeClassifier  model 시작
	 폴드 세트:  0 시작
	 폴드 세트:  1 시작
	 폴드 세트:  2 시작
	 폴드 세트:  3 시작
	 폴드 세트:  4 시작
	 폴드 세트:  5 시작
	 폴드 세트:  6 시작


In [31]:
# AdaBoost
ada_train, ada_test = get_stacking_base_datasets(
      ada_clf
    , X_train
    , y_train
    , X_test
    , 7
)

AdaBoostClassifier  model 시작
	 폴드 세트:  0 시작
	 폴드 세트:  1 시작
	 폴드 세트:  2 시작
	 폴드 세트:  3 시작
	 폴드 세트:  4 시작
	 폴드 세트:  5 시작
	 폴드 세트:  6 시작


In [34]:
# knn, rf, dt, ada 데이터 Stacking
stack_X_train = np.concatenate(
      (knn_train,rf_train,dt_train,ada_train)
    , axis=1
)

stack_X_test = np.concatenate(
      (knn_test,rf_test,dt_test,ada_test)
    , axis=1
)

In [35]:
# 원래 학습 데이터 : X_train, 원래 테스트 데이터 : X_test
X_train.shape, X_test.shape

((455, 30), (114, 30))

In [36]:
# 스태킹 처리된 학습/테스트 데이터
stack_X_train.shape, stack_X_test.shape

((455, 4), (114, 4))

### 최종 모델 학습 생성

In [38]:
lr_clf.fit(stack_X_train, y_train)

# 예측값 추출
stack_pred = lr_clf.predict(stack_X_test)

stack_accu = accuracy_score(y_test, stack_pred)
print('최종 정확도 : {0:.4f}'.format(stack_accu))

최종 정확도 : 0.9737
